* * *
<pre> NYU Paris            <i> Artificial intelligence - Fall 2023 </i></pre>
* * *


<h1 align="center"> Lab 7: Introduction to deep learning </h1>

<pre align="left"> October 12th 2023               <i> Author: Hicham Janati </i></pre>
* * *


##### Goals:
- Discover pytorch
- Create your own neural network
- Adapt the complexity of the neural network

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import torch
from torch import nn, optim
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_lfw_people
from sklearn.decomposition import PCA

people = fetch_lfw_people(min_faces_per_person=60, resize=0.7)
image_shape = people.images[0].shape
X = people.images.reshape(len(people.images), -1)
y = people.target
X = X / 255

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# split the data in training and test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=0)

pca = PCA(100)
pca.fit(X_train)

X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)

y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

print(X_train.shape, X_test.shape, len(np.unique(y)))


With pytorch, a neural netwrok is an boject that inherist from `nn.Module`. It must contain:

- a constructor ```___init___``` which specifies its attributes (number of layers, neurones etc)
- a forward function ```forward``` that computes the output of the neural net given a data input `X`.

In [ ]:
import torch
from sklearn import preprocessing
from sklearn.metrics import classification_report


class NeuralNet(nn.Module):
    """Reseau de neurones simple."""

    def __init__(self):
        super(NeuralNet, self).__init__()
        self.linear_layer = nn.Linear(100, 8) 

    def forward(self, X, **kwargs):
        X = self.linear_layer(X)
        
        torch.nn.LogSoftmax(dim=-1)(X).squeeze()
        return X


### Question 1
What does this first neural net above correspond to ?

After creating our neural network, we need to write a function that performs the 'fit' operation i.e solve the optimization problem.

In [ ]:
def fit_neural_net(neural_net,
                   X_train, X_test, y_train, y_test,
                   learning_rate=0.005,
                   weight_decay=0.,
                   max_iter=1000,
                   device='cpu',
                   verbose=True):
    """
    Function that performs back-propagation (gradient descent)
    to optimize the neural net.
    neural_net: Instance of NeuralNet
    learning_rate: gradient step size
    weight_decay: regularization parameter of L2 regularization
    max_iter: nombre maximum d'itérations
    device: 'cpu' or 'cuda:0'

    Returns
    scores: dictionary of s
    """

    loss_func = nn.CrossEn().to(device)
    neural_net = neural_net.to(device)

    optimizer = optim.Adam(neural_net.parameters(), lr=learning_rate, weight_decay=weight_decay)

    loss_train, loss_test = [], []
    accuracy_train, accuracy_test = [], []

    if verbose:
        strings = ["Iteration", "Accuracy Train", "Accuracy Test"]
        strings = [s.center(13) for s in strings]
        strings = " | ".join(strings)
        print(strings)

    for ii in range(max_iter):
        # Train
        neural_net.train()
        optimizer.zero_grad()
        log_proba_pred_train = neural_net(X_train)
        loss_train_ = loss_func(log_proba_pred_train, y_train)  #### compute the loss
        loss_train_.backward()  ### compute the gradients and do back propagation
        optimizer.step()    # gradient step


      # Test
      # turn neural net to evaluation mode to not keep computing gradients
        neural_net.eval()
        with torch.no_grad():
            log_proba_pred_test = neural_net(X_test)
            y_pred_train = torch.exp(log_proba_pred_train).argmax(1) ### On passes des log probas de chaque classe a une prediction
            y_pred_test = torch.exp(log_proba_pred_test).argmax(1)
            loss_test_ = loss_func(log_proba_pred_test, y_test).item()  #### On calcule la perte
            accuracy_test_ = (y_pred_test == y_test).float().mean().item() #### On calcule la precision test
            accuracy_train_ = (y_pred_train == y_train).float().mean().item() #### On calcule la precision train

        if verbose and ii % 50 == 0:
            strings = [ii, accuracy_train_, accuracy_test_] # On affiche des trucs 
            strings = [str(s).center(14) for s in strings]
            strings = " | ".join(strings)
            print(strings)

        loss_train.append(loss_train_.item())
        loss_test.append(loss_test_)
        accuracy_train.append(accuracy_train_)
        accuracy_test.append(accuracy_test_)

    metrics = [[loss_train, loss_test],
              [accuracy_train, accuracy_test]]
    return metrics


def make_report(neural_net, X_test):
    neural_net.eval()
    y_pred = neural_net(X_test).detach().cpu().numpy().argmax(1)
    print(classification_report(y_test.cpu().numpy(), y_pred))

neural_net = NeuralNet()
metrics = fit_neural_net(neural_net, X_train, X_test, y_train, y_test)
make_report(neural_net, X_test)

### Question 2
Display the loss and accuracy of both train and test as a function of the training epochs (iterations).

### Question 3
Modify the neural net above and create a more sophisticated one with 1 hidden layer with a ReLU activation function containing 20 neurones. Does the performance improve ?

In [ ]:
import torch
from sklearn import preprocessing
from sklearn.metrics import classification_report


class NeuralNet_v2(nn.Module):
    """Neural net with 1 hidden layer ."""

    def __init__(self):
        super(NeuralNet, self).__init__()
        # to do 
        
    def forward(self, X, **kwargs):
        X = self.linear_layer(X)
        # todo 
        X = torch.nn.LogSoftmax(dim=-1)(X).squeeze()
        return X


### Question 4
The following net is a specific neural net known as convolutional neural networks. Train the model and see if the performance is better. Explain.

In [ ]:
class ConvNet(nn.Module):
    """Basic ConvNet"""

    def __init__(self, n_outputs=10, debug=False):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(2304, 100)
        self.fc2 = nn.Linear(100, n_outputs)
        self.maxpool = nn.MaxPool2d(2)
        self.debug = debug
        

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = torch.relu(x)
        x = self.conv2(x)
        # x = self.conv2_drop(x)
        x = self.maxpool(x)
        x = torch.relu(x)

        # flatten over channel, height and width = 1600
        x = x.view(-1, x.size(1) * x.size(2) * x.size(3))

        if self.debug:  # trick to get the size of the first FC
            print("### DEBUG: Shape of last convnet=", x.shape,
                  ". FC size=", np.prod(x.shape[1:]))

        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = torch.nn.LogSoftmax(dim=-1)(x).squeeze()
        return x

loss_func = nn.NLLLoss()
conv_net = ConvNet()
optimizer = optim.Adam(conv_net.parameters(), lr=0.001, weight_decay=0.0001)

scaler = preprocessing.MinMaxScaler()
X_train_s = torch.tensor(scaler.fit_transform(X_train)).reshape(-1, 1, 32, 32)
X_test_s = torch.tensor(scaler.transform(X_test)).reshape(-1, 1, 32, 32)
y_train = torch.tensor(y_train)
n_iters = 0

losses = []
for ii in range(n_iters):
    optimizer.zero_grad()
    y_pred_train = conv_net(X_train_s)
    risque = loss_func(y_pred_train, y_train)  #### On calcule la perte
    losses.append(risque.detach())
    risque.backward()  ### On calcule les gradients avec la back-propagation
    optimizer.step()    # pas descente de gradient
    

conv_net.eval()
y_pred = conv_net(X_test_s).detach().cpu().numpy().argmax(1)
print(classification_report(y_test, y_pred))